# 1. Churn Dataset

## 1.1 Import

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style("darkgrid")

from IPython.display import display, Markdown
pd.set_option('display.max_columns', None)  

DEBUG = False

import os
for d in ['src','data','output']: os.makedirs(d, exist_ok=True)

In [28]:
for filename in ['churn.csv','states.csv']:
    source = f"https://kmurphy.bitbucket.io/modules/Data_Mining_2/topics/01-Module_Introduction/20-Practical_01_-_Review_of_Pandas_-_Churn/files/{filename}"
    target = f"src/{filename}"

    if not os.path.isfile(target):
        print (f"Downloading remote file {filename}", sep="")
        import urllib.request
        urllib.request.urlretrieve(source, target)
    else:
        print(f"Using local copy of {filename}")

Using local copy of churn.csv
Using local copy of states.csv


## 1.2 Clean

In [29]:
churn_dataset = pd.read_csv('src/churn.csv')
state_dataset = pd.read_csv('src/states.csv')

# make copy of dataset
churn_cleaned = churn_dataset
state_cleaned = state_dataset

In [30]:
cleaned_churn_names = []

for col in churn_cleaned.columns:

    col = col.replace(' ', '_')
    col = col.replace("'", '')
    col = col.replace('?', '')
        
    col = col.title()
    cleaned_churn_names.append(col)
    
churn_cleaned.columns = cleaned_churn_names
state_cleaned.columns = [x.title() for x in state_cleaned.columns]

## 1.3 Merge

In [31]:
churn_merged = pd.merge(churn_cleaned, state_cleaned, on="State")

## 1.4 Analysis

In [32]:
# find state with largest account length
print('\n\n State with largest account \n\n', 
      churn_merged.groupby(['State']).mean().sort_values(by=['Account_Length'], ascending=False)['Account_Length'].head(1))

# find state with third largest customer length
print('\n\n State with third largest customer length \n\n', churn_merged['State'].value_counts().head(3))

# find customers with both international and voicemail plans
print('\n\n Customers with both international and voicemal plans \n\n', 
      len(churn_merged[(churn_merged['Intl_Plan']=='yes') & (churn_merged['Vmail_Plan']=='yes')]))

# find percent of customers with 0 customer service calls
print('\n\n Percent of customers with 0 CS calls \n\n', 
      (len(churn_merged[(churn_merged['Custserv_Calls']==0)]) / len(churn_merged)) * 100)

# find the mean account length for customers that churned
churned_cust = churn_merged[(churn_merged['Churn']=="True.")]
print('\n\n Mean account length of customers that churned \n\n',  churned_cust['Account_Length'].mean())

# find the mean account length for customers that have not churned
not_churned_cust = churn_merged[(churn_merged['Churn']=="False.")]
print('\n\n Mean account length for customers that have not churned \n\n', not_churned_cust['Account_Length'].mean())

# find mean account length
print('\n\nMean account length \n\n', churn_merged['Account_Length'].mean())

# find mean number of voice mail messages for customers with a voicemail plan
voice_mail = churn_merged[(churn_merged['Vmail_Plan'] == 'yes')]
print('\n\n Mean number of voice mail messages for customers with a voicemail plan\n\n', voice_mail.mean())

# find median account length
print('\n\n Median account length \n\n', churn_merged['Account_Length'].median())

# find percentage of customers with voice mail plans also have international plans
print('\n\n Percentage of customers with voice mail plans that also have international plans \n\n', 
      (len(churn_merged[(churn_merged['Intl_Plan']=='yes') & (churn_merged['Vmail_Plan']=='yes')]) 
      / len(churn_merged[(churn_merged['Vmail_Plan']=='yes')])) * 100)

# is average account length for churning customers is longer than that for non-churning customers
churned_account_mean = churn_merged['Account_Length'][(churn_merged['Churn']=="True.")].mean()
not_churned_account_mean = churn_merged['Account_Length'][(churn_merged['Churn']=="False.")].mean()
print('\n\n Is average account length for churning customers longer than non-churning customers? \n\n', 
      churned_account_mean > not_churned_account_mean)

# find how many states have a customer churn rate greater or equal to 20%
churn_backup = churn_merged
churn_backup.loc[churn_backup.Churn=='False.','Churn'] = 0 
churn_backup.loc[churn_backup.Churn=='True.','Churn'] = 1
print('\n\n Staets with churn rate greater than or equal to 20% \n\n')
for index, val in (churn_backup.groupby('State').Churn.mean() * 100).items():
    if val >= 20: 
        print(index)
    
# of largest 20 states in terms of number of customers, find the range of the mean account length per state
states = list(churn_merged.State.value_counts().nlargest(20).index)
top_20 = churn_merged.loc[(churn_merged.State).isin(states)]
sorted_top_20 =  top_20.groupby(['State']).mean().sort_values(by=['Account_Length'], ascending=False)['Account_Length']
print('\n\n Of largest 20 states, what is the range of the mean account length per state \n\n',
      sorted_top_20.max() - sorted_top_20.min())

# within the 5 states with largest median account length, what is the mean account length?
median_states = list(churn_merged.groupby('State').median()['Account_Length'].nlargest(5).index)
top_5 = churn_merged.loc[(churn_merged.State).isin(median_states)]
print('\n\n Within the 5 states with the largest median account length, what is the mean account length \n\n', 
      top_5.groupby(['State']).mean()['Account_Length'].sum() / len(top_5.groupby(['State'])))



 State with largest account 

 State
FL    109.571429
Name: Account_Length, dtype: float64


 State with third largest customer length 

 WV    106
MN     84
NY     83
Name: State, dtype: int64


 Customers with both international and voicemal plans 

 92


 Percent of customers with 0 CS calls 

 20.912091209120913


 Mean account length of customers that churned 

 102.66459627329192


 Mean account length for customers that have not churned 

 100.79368421052632


Mean account length 

 101.06480648064806


 Mean number of voice mail messages for customers with a voicemail plan

 Account_Length    101.252711
Area_Code         437.131236
Vmail_Message      29.277657
Day_Mins          179.626790
Day_Calls         100.075922
Day_Charge         30.537072
Eve_Mins          202.746963
Eve_Calls          99.906725
Eve_Charge         17.233796
Night_Mins        201.369089
Night_Calls       100.599783
Night_Charge        9.061638
Intl_Mins          10.231345
Intl_Calls          4.509761
In